In [1]:
import pandas as pd
pd.set_option('display.min_rows', 200)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', None)  # Adjusts width to fit all columns
pd.set_option('display.max_colwidth', None)  # Prevents column content truncation

## Obróbka wyborów europejskich ##

In [2]:
euro_df = pd.read_csv("dane/parlament_euro.csv", sep=";")
euro_df.head(1)

,TERYT Gminy,Gmina,Powiat,Województwo,Liczba komisji,Liczba uwzględnionych komisji,SIENKIEWICZ Bartłomiej Henryk - KKW KOALICJA OBYWATELSKA,WILK-GRZYWNA Małgorzata Beata - KKW KOALICJA OBYWATELSKA,MARCZUŁAJTIS-WALCZAK Jagna Kinga - KKW KOALICJA OBYWATELSKA,SALWIŃSKA Sylwia Kinga - KKW KOALICJA OBYWATELSKA,...,SKUZA Stanisław - KWW BEZPARTYJNI SAMORZĄDOWCY-NORMALNA POLSKA,NOWAKOWSKA Elżbieta Paula - KWW BEZPARTYJNI SAMORZĄDOWCY-NORMALNA POLSKA,WICHE Mateusz Andrzej - KWW BEZPARTYJNI SAMORZĄDOWCY-NORMALNA POLSKA,KALISZ Beata - KWW BEZPARTYJNI SAMORZĄDOWCY-NORMALNA POLSKA,TORKOWSKA Samuela - KWW BEZPARTYJNI SAMORZĄDOWCY-NORMALNA POLSKA,GOCAŁ Dariusz Krzysztof - KWW BEZPARTYJNI SAMORZĄDOWCY-NORMALNA POLSKA,SIEŃKO Natalia Izabela - KWW BEZPARTYJNI SAMORZĄDOWCY-NORMALNA POLSKA,WOCH Marcin Grzegorz - KWW BEZPARTYJNI SAMORZĄDOWCY-NORMALNA POLSKA,BUGAJSKI Marcin Rafał - KWW BEZPARTYJNI SAMORZĄDOWCY-NORMALNA POLSKA,PETERSON Magdalena - KWW BEZPARTYJNI SAMORZĄDOWCY-NORMALNA POLSKA
0,120101,m. Bochnia,bocheński,małopolskie,18,18,2247,106,409,41,...,11,21,3,5,5,4,1,2,10,3


In [3]:
euro_df_tylko_kandydaci = euro_df.iloc[:, 6:]

In [4]:
euro_razem_wszystkie_glosy = euro_df["ŻAK Dominik Piotr - KKW LEWICA"] + euro_df["KOLARSKA Dorota Anna - KKW LEWICA"]
euro_razem_wszystkie_glosy = euro_razem_wszystkie_glosy.sum()
euro_razem_wszystkie_glosy
euro_df["glosy_razem"] = euro_df["ŻAK Dominik Piotr - KKW LEWICA"] + euro_df["KOLARSKA Dorota Anna - KKW LEWICA"]
euro_df["glosy_wszytkie"] = euro_df_tylko_kandydaci.sum(axis=1)
euro_df["procent_razem"] = euro_df["glosy_razem"] / euro_df["glosy_wszytkie"] * 100

In [5]:
euro_wszystkie_glosy = euro_df_tylko_kandydaci.sum().sum()
euro_okregowy_procent_na_razem = euro_razem_wszystkie_glosy / euro_wszystkie_glosy * 100
euro_okregowy_procent_na_razem

np.float64(2.0805401585761993)

In [6]:

wynik_proc_adriana = 4.86
alpha_euro = wynik_proc_adriana / euro_okregowy_procent_na_razem
alpha_euro
# wielkość przez którą domnażamy wynik w wyborach europejskich (2.08%) tak, żeby był porównywalny z wynikami prezydenckimi

np.float64(2.335931839607413)

In [7]:
euro_df["procent_razem_od_sredniej"] = euro_df["procent_razem"] - euro_okregowy_procent_na_razem
euro_df["znormalizowany_procent_razem_od_sredniej"] = euro_df["procent_razem_od_sredniej"] * alpha_euro

## Wybory prezydenckie ##

In [8]:

prezydent_df = pd.read_csv("dane/prezydent_gminy.csv", sep=";")

In [9]:

ogolnopolski_procent_na_adriana = 4.86
prezydent_df["procent_na_adriana"] = prezydent_df["ZANDBERG Adrian Tadeusz"] / prezydent_df["Liczba głosów ważnych oddanych łącznie na wszystkich kandydatów"] * 100
prezydent_df["procent_na_adriana_od_sredniej"] = prezydent_df["procent_na_adriana"] - ogolnopolski_procent_na_adriana

## Łączenie tabel i "wielowyborczy" wskaźnik ##

In [10]:
euro_prez_df = euro_df.merge(prezydent_df, on="TERYT Gminy", suffixes=["_euro", "_prez"], how="left")

In [11]:
waga_euro = .5
waga_prez = .5
# to są wagi, którymi domnażamy wynik w poszczególnych wyborach. 
# Jeśli interesuje nas tylko poparcie w wyborach prezydenckich należy ustawić waga_prez = 1, waga_euro = 0.
# Analogicznie jeśli interesują nas tylko eurowybory

euro_prez_df["wielowyborczy_procent_od_sredniej"] = waga_euro * euro_prez_df["znormalizowany_procent_razem_od_sredniej"] + waga_prez * euro_prez_df["procent_na_adriana_od_sredniej"]
sorted_df = euro_prez_df.sort_values("wielowyborczy_procent_od_sredniej", ascending=False)


In [12]:
wynik_df = sorted_df[["Gmina_prez", "Powiat_prez", "Województwo_prez", "wielowyborczy_procent_od_sredniej", "znormalizowany_procent_razem_od_sredniej", "procent_na_adriana_od_sredniej"]]
wynik_df.columns = ["Gmina", "Powiat", "Województwo", "Wielowyborczy wskaźnik poparcia", "Zormalizowany % Euro minus średnia", "% prezydent minus średnia"]
wynik_df

,Gmina,Powiat,Województwo,Wielowyborczy wskaźnik poparcia,Zormalizowany % Euro minus średnia,% prezydent minus średnia
280,Kraków,Kraków,małopolskie,7.995997,10.213874,5.778119
54,gm. Zielonki,krakowski,małopolskie,2.072344,2.919704,1.224983
52,gm. Wielka Wieś,krakowski,małopolskie,1.589862,2.263824,0.915899
53,gm. Zabierzów,krakowski,małopolskie,0.904532,1.783648,0.025415
45,gm. Michałowice,krakowski,małopolskie,0.780696,1.261300,0.300091
177,gm. Niepołomice,wielicki,małopolskie,0.718543,1.008019,0.429067
178,gm. Wieliczka,wielicki,małopolskie,0.662728,0.675785,0.649672
43,gm. Krzeszowice,krakowski,małopolskie,0.208027,0.192500,0.223553
48,gm. Skawina,krakowski,małopolskie,-0.023526,-0.188136,0.141083
51,gm. Świątniki Górne,krakowski,małopolskie,-0.091151,-0.426750,0.244447


In [ ]:
wynik_df.to_excel("prezydent_euro_gminy.xlsx", index=False, float_format="%.2f")